In [ ]:
import plotly.graph_objects as go
import pymesh

%load_ext autoreload
%autoreload 2

In [ ]:
mesh = pymesh.load_mesh("data/3dmodels/landscape_example.obj")

In [ ]:
mesh.num_vertices

In [ ]:
mesh.faces

In [ ]:
# Plot point cloud
fig = go.Figure(data=[go.Scatter3d(
    x=mesh.vertices[:, 0],
    y=mesh.vertices[:, 1],
    z=mesh.vertices[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=mesh.vertices[:, 2],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])
fig.update_layout(width=1600, height=900)
fig.update_layout(scene = dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot mesh
fig = go.Figure(data=[go.Mesh3d(x=mesh.vertices[:,0],
                                y=mesh.vertices[:,1],
                                z=mesh.vertices[:,2],
                                i=mesh.faces[:,0],
                                j=mesh.faces[:,1],
                                k=mesh.faces[:,2],
                                color='lightpink',
                                opacity=0.50)])
# Set figsize
fig.update_layout(width=1600, height=900)
fig.update_layout(scene = dict(aspectmode='data'))
fig.show()